# Tasks

What is a Julia Task?

- very lightweight coroutines
- Not threads!
- Internal to and scheduled by a Julia Process

In [1]:
?Task

search: Task task_local_storage @task istaskdone istaskstarted current_task



```
Task(func)
```

Create a `Task` (i.e. coroutine) to execute the given function (which must be callable with no arguments). The task exits when this function returns.

```jldoctest
julia> a() = det(rand(1000, 1000));

julia> b = Task(a);
```

In this example, `b` is a runnable `Task` that hasn't started yet.


In [6]:
function mytask()
    println("Going to take a nap.")
    sleep(10)
    println("Woke up.")
    rand()
end

t=Task(mytask)

Task (runnable) @0x0000000123fc2410

## Scheduling a task

**`schedule`** actually starts the task, but will *return immediately*

In [7]:
schedule(t)

Going to take a nap.


Task (runnable) @0x0000000123fc2410

## Waiting on a task

In [8]:
println("Doing something else while t is taking a nap...")
inv(rand(100, 100))
@time @show wait(t)
@show t.state
println("task finished")

Doing something else while t is taking a nap...
Woke up.
wait(t) = 0.29036133652429497
  9.050541 seconds (7.31 k allocations: 408.367 KiB)
t.state = :done
task finished


## @async - syntax sugar for creating tasks and scheduling it

In [9]:
t=@async begin
    println("Going to take a nap.")
    sleep(5)
    println("Woke up.")
end

Going to take a nap.


Task (runnable) @0x0000000123c27190

In [10]:
21+21

42

Woke up.


## Channels

Allows communication between Tasks

In [26]:
input = Channel{Int}(1)
result = Channel{Int}(1)
doubler = @async while true
    x = take!(input)
    println("Got message $x")
    put!(result, 2x)
end

printer = @async while true
    res = take!(result)
    @show res
end

Task (runnable) @0x0000000125430d90

In [27]:
using Interact

In [28]:
@manipulate for i=1:100
    put!(input, i)
end

Interact.Options{:SelectionSlider,Any}(21: "input-6" = 50 Any , "i", 50, "50", 50, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…), Dict{Any,Any}(Pair{Any,Any}(68, "68"),Pair{Any,Any}(2, "2"),Pair{Any,Any}(89, "89"),Pair{Any,Any}(11, "11"),Pair{Any,Any}(39, "39"),Pair{Any,Any}(46, "46"),Pair{Any,Any}(85, "85"),Pair{Any,Any}(25, "25"),Pair{Any,Any}(55, "55"),Pair{Any,Any}(42, "42")…)), Any[], Any[], true, "horizontal")

Got message 50


50

res = 100
Got message 46
res = 92
Got message 45
res = 90
Got message 44
res = 88
Got message 42
res = 84
Got message 40
res = 80
Got message 30
res = 60
Got message 31
res = 62
Got message 32
res = 64
Got message 33
res = 66
Got message 58
res = 116
Got message 61
res = 122
Got message 62
res = 124


## Adding Julia Processes, running "Remote Tasks"

In [29]:
# Run if using the notebook on your own computer
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [30]:
procs()

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

## Estimate pi in parallel

In [31]:
@everywhere function trials(numsteps=1000)  # default value of the parameter
    pos = 0 
    for j in 1:numsteps
        pos += Int(rand()^2 + rand()^2 < 1)
    end
    return pos
end

function estimate_pi(in_circle, N)
    4in_circle / N
end

estimate_pi (generic function with 1 method)

In [32]:
estimate_pi(trials(10^8), 10^8)

3.14129036

In [33]:
# @spawnat is like @async but runs on a different process
f=@spawnat 3 begin
    println("Process ", myid(), " starting random trials")
    res = trials(10^8)
    println("Process ", myid(), " done")
    res
end

Future(3, 1, 10, Nullable{Any}())

	From worker 3:	Process 3 starting random trials
	From worker 3:	Process 3 done


In [34]:
typeof(f)

Future

What's the curious `Future(3,1,12,Nullable{Any}())` thing?

In [35]:
f[]

78532694

In [36]:
# @spawnat is like @async but runs on a different process
function remote_trials(pid,n)
    @spawnat pid begin
        println("Process ", myid(), " starting trials")
        trials(n)
    end
end

remote_trials (generic function with 1 method)

In [37]:
remote_trials(2, 1000)

Future(2, 1, 12, Nullable{Any}())

	From worker 2:	Process 2 starting trials


In [38]:
function parallel_trials(n, pids=workers())
    @time futures = [remote_trials(p,n) for p in pids]
    sum([f[] for f in futures])
end

parallel_trials (generic function with 2 methods)

In [39]:
@time estimate_pi(parallel_trials(10^8), 10^8*nworkers())

	From worker 2:	Process 2 starting trials
  0.025225 seconds (1.32 k allocations: 73.993 KiB)
	From worker 3:	Process 3 starting trials
	From worker 5:	Process 5 starting trials
	From worker 4:	Process 4 starting trials
  1.573160 seconds (50.47 k allocations: 2.902 MiB, 0.70% gc time)


3.14163855

In [40]:
@time estimate_pi(trials(10^8*nworkers()), 10^8*nworkers())

  1.884120 seconds (18 allocations: 384 bytes)


3.14155212